<a href="https://colab.research.google.com/github/abdyraman/internet-customers/blob/main/internet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.impute import SimpleImputer

In [26]:
df=pd.read_csv('internet_service_churn.csv')
df

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
72269,1689648,1,1,0.09,0,1.25,0,0.0,0.0,0,1
72270,1689676,1,0,0.06,1,1.63,0,0.8,0.0,0,1
72271,1689679,1,0,0.02,0,2.19,0,1.5,0.2,0,1
72272,1689733,0,0,0.01,0,0.72,0,0.0,0.0,0,1


Data cleaning

In [27]:
# Determine the number of unique values in each column.
unique_counts = df.nunique()
unique_counts

id                             72274
is_tv_subscriber                   2
is_movie_package_subscriber        2
subscription_age                1110
bill_avg                         179
reamining_contract               247
service_failure_count             19
download_avg                    2856
upload_avg                       802
download_over_limit                8
churn                              2
dtype: int64

In [39]:
# Check for NaN values in the DataFrame
nan_counts = df.isnull().sum()
# Determine the number of unique values in each column
unique_counts = df.nunique()
# Print the NaN counts and unique value counts for each column
print("NaN counts for each column:\n", nan_counts)

NaN counts for each column:
 id                                 0
is_tv_subscriber                   0
is_movie_package_subscriber        0
subscription_age                   0
bill_avg                           0
reamining_contract             21572
service_failure_count              0
download_avg                     381
upload_avg                       381
download_over_limit                0
churn                              0
dtype: int64


In [29]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cleaned_df = df.drop(['id','reamining_contract'], axis=1)
cleaned_df

,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,1,0,11.95,25,0,8.4,2.3,0,0
1,0,0,8.22,0,0,0.0,0.0,0,1
2,1,0,8.91,16,0,13.7,0.9,0,1
3,0,0,6.87,21,1,0.0,0.0,0,1
4,0,0,6.39,0,0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...
72269,1,1,0.09,0,0,0.0,0.0,0,1
72270,1,0,0.06,1,0,0.8,0.0,0,1
72271,1,0,0.02,0,0,1.5,0.2,0,1
72272,0,0,0.01,0,0,0.0,0.0,0,1


In [30]:
# Initialize the imputer with the most frequent strategy
final_imputer = SimpleImputer(strategy='most_frequent')
# Fit and transform the DataFrame with the imputer
preprocessed_df = pd.DataFrame(final_imputer.fit_transform(cleaned_df), columns=cleaned_df.columns)
# Determine the number of unique values in each column in preprocessed df
unique_counts = preprocessed_df.nunique()
unique_counts

is_tv_subscriber                  2
is_movie_package_subscriber       2
subscription_age               1110
bill_avg                        179
service_failure_count            19
download_avg                   2856
upload_avg                      802
download_over_limit               8
churn                             2
dtype: int64

In [41]:
# Check for NaN values in the DataFrame
nan_counts = preprocessed_df.isnull().sum()
# Determine the number of unique values in each column
unique_counts = preprocessed_df.nunique()
# Print the NaN counts and unique value counts for each column
print("NaN counts for each column:\n", nan_counts)

NaN counts for each column:
 is_tv_subscriber               0
is_movie_package_subscriber    0
subscription_age               0
bill_avg                       0
service_failure_count          0
download_avg                   0
upload_avg                     0
download_over_limit            0
churn                          0
dtype: int64


There is a big competition between Internet providers. If a providers want to increase its revenue they needs more subscriber but keep existing customer is more important than having new ones. So providers want to know which customer should cancel his service. we call this as churn. if the know who will go, maybe they can catch them with promotions.

In [42]:
# Remove target from features data
y = preprocessed_df.churn.values
X = preprocessed_df.drop(columns="churn").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [32]:
# Preprocess numerical data for neural network
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=8))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [34]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 15.5 MB/s eta 0:00:00


In [44]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [45]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 02m 24s]
val_accuracy: 0.7396093010902405

Best val_accuracy So Far: 0.7983286380767822
Total elapsed time: 00h 44m 00s


In [46]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 21, 'num_layers': 2, 'units_0': 26, 'units_1': 26, 'units_2': 26, 'units_3': 11, 'units_4': 26, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0038'}
{'activation': 'relu', 'first_units': 26, 'num_layers': 5, 'units_0': 11, 'units_1': 6, 'units_2': 21, 'units_3': 11, 'units_4': 11, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 26, 'num_layers': 4, 'units_0': 21, 'units_1': 1, 'units_2': 26, 'units_3': 26, 'units_4': 21, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}


In [47]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

565/565 - 1s - loss: 0.4297 - accuracy: 0.7983 - 1s/epoch - 2ms/step
Loss: 0.42969459295272827, Accuracy: 0.7983286380767822
565/565 - 1s - loss: 0.4321 - accuracy: 0.7972 - 1s/epoch - 2ms/step
Loss: 0.4320782721042633, Accuracy: 0.7972217798233032
565/565 - 1s - loss: 0.4343 - accuracy: 0.7969 - 935ms/epoch - 2ms/step
Loss: 0.4342982769012451, Accuracy: 0.7968897223472595


In [48]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'relu',
 'first_units': 26,
 'num_layers': 5,
 'units_0': 11,
 'units_1': 6,
 'units_2': 21,
 'units_3': 11,
 'units_4': 11,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [49]:
# Compare the performance to the second-best model
second_model = tuner.get_best_models(2)[1]
model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

565/565 - 1s - loss: 0.4321 - accuracy: 0.7972 - 1s/epoch - 3ms/step
Loss: 0.4320782721042633, Accuracy: 0.7972217798233032


Alternative solution

In [24]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
1694/1694 [==============================] - 4s 2ms/step - loss: 0.5129 - accuracy: 0.7402
Epoch 2/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4795 - accuracy: 0.7634
Epoch 3/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4711 - accuracy: 0.7669
Epoch 4/50
1694/1694 [==============================] - 4s 2ms/step - loss: 0.4671 - accuracy: 0.7689
Epoch 5/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4639 - accuracy: 0.7710
Epoch 6/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4615 - accuracy: 0.7730
Epoch 7/50
1694/1694 [==============================] - 4s 2ms/step - loss: 0.4603 - accuracy: 0.7728
Epoch 8/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4588 - accuracy: 0.7738
Epoch 9/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.4576 - accuracy: 0.7747
Epoch 10/50
1694/1694 [==============================] - 3s 2ms/step - loss: 0.456